# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

### Train-test split

Perform a train-test split with a test set of 20%.

In [3]:
# Import train_test_split from sklearn.model_selection
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [ ]:
# Split the data into training and test sets (assign 20% to test set)


In [4]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))


1168 292 1168 292


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [5]:
# Your code here
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [6]:
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [8]:
# Your code here
from sklearn.metrics import mean_squared_error
train_residuals = y_train - y_hat_train
test_residuals = y_test - y_hat_test
train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)

print('Training MSE: ', train_mse)
print('Testing MSE: ', test_mse)

Training MSE:  0.16463869420483296
Testing MSE:  0.15676157082417494


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [40]:
df = pd.DataFrame(preprocessed)
df

,LotArea_log,1stFlrSF_log,GrLivArea_log,SalePrice_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,...,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker
0,-0.133185,-0.803295,0.529078,0.559876,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.113403,0.418442,-0.381715,0.212692,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.419917,-0.576363,0.659449,0.733795,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.103311,-0.439137,0.541326,-0.437232,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.878108,0.112229,1.281751,1.014303,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.259100,-0.465447,0.416538,0.121392,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1456,0.725171,1.980456,1.106213,0.577822,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1457,-0.002324,0.228260,1.469438,1.174306,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1458,0.136814,-0.077546,-0.854179,-0.399519,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [38]:
len(df)//6

243

In [28]:
k = 6
q = len(df)//k
r = len(df)%k
# print('Quotient: ', q,'\nRemainder: ',r)
folds = []
starting_index = 0
for fold_n in range(k):
    if r > 0:
        folds.append(df.iloc[starting_index:starting_index+q+1,:])
        starting_index += q + 1
        r -= 1
    else:
        folds.append(df.iloc[starting_index:starting_index+q,:])
        starting_index += q

Quotient:  243 
Remainder:  2


In [43]:
# for k in range(len(folds)):
#     print(k,len(folds[k]))
#     display(folds[k].head(1))
#     display(folds[k].tail(1))
#     print('-'*35)

In [48]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    df = pd.DataFrame(data)
    q = len(df)//k
    r = len(df)%k
#     print('Quotient: ', q,'\nRemainder: ',r)
    folds = []
    starting_index = 0
    for fold_n in range(k):
        if r > 0:
            folds.append(df.iloc[starting_index:starting_index+q+1,:])
            starting_index += q + 1
            r -= 1
        else:
            folds.append(df.iloc[starting_index:starting_index+q,:])
            starting_index += q
    # add 1 to fold size to account for leftovers           
    return folds

### Apply it to the Ames Housing data

In [70]:
# Make sure to concatenate the data again
ames_data = pd.concat([X, y], axis=1)
ames_data

,LotArea_log,1stFlrSF_log,GrLivArea_log,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,...,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,SalePrice_log
0,-0.133185,-0.803295,0.529078,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.559876
1,0.113403,0.418442,-0.381715,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0.212692
2,0.419917,-0.576363,0.659449,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.733795
3,0.103311,-0.439137,0.541326,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-0.437232
4,0.878108,0.112229,1.281751,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.014303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-0.259100,-0.465447,0.416538,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.121392
1456,0.725171,1.980456,1.106213,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0.577822
1457,-0.002324,0.228260,1.469438,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.174306
1458,0.136814,-0.077546,-0.854179,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,-0.399519


In [71]:
# Apply kfolds() to ames_data with 5 folds
folds = kfolds(ames_data, k=5)

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [72]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    train = pd.concat([x for i,x in enumerate(folds) if i!=n])
    test = folds[n]

    X_train = train.drop('SalePrice_log', axis=1)
    y_train = train['SalePrice_log']

    linreg.fit(X_train, y_train)

    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    
    train_resid = y_hat_train - y_train
    test_resid = y_hat_test - y_test
    
#     train_errs.append(np.mean(train_resid))
#     test_errs.append(np.mean(test_resid))
    train_errs.append(mean_squared_error(y_train, y_hat_train))
    test_errs.append(mean_squared_error(y_test, y_hat_test))

print(train_errs)
print(test_errs)

[0.1717050965146466, 0.15507935685930538, 0.15659946326223304, 0.16134557666308721, 0.1516504855313168]
[0.14146802903163574, 0.14211105854506323, 0.14162656050130704, 0.135971123299596, 0.1407449625361144]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [73]:
# Your code here
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score

mse = make_scorer(mean_squared_error)

print(cross_val_score(linreg, X, y, cv=5, scoring=mse))

[0.12431546 0.19350065 0.1891053  0.17079325 0.20742705]


Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [65]:
# Your code here
cv_5_results.mean()

0.1770283421000112

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 